In [1]:
import os
import torch
import sys

sys.path.append('/home/sxk230060/TI2AV/misc/ltx_lora_training_i2v_t2v')
from ltx_video_lora import load_latent_models

/home/sxk230060/miniconda3/envs/ltx_audio/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda"
dtype = torch.bfloat16
vae = load_latent_models()["vae"].to(device, dtype=dtype)

In [3]:
def _unpack_latents(
        latents: torch.Tensor, num_frames: int, height: int, width: int, patch_size: int = 1, patch_size_t: int = 1
    ) -> torch.Tensor:
    batch_size = latents.size(0)
    latents = latents.reshape(batch_size, num_frames, height, width, -1, patch_size_t, patch_size, patch_size)
    latents = latents.permute(0, 4, 1, 5, 2, 6, 3, 7).flatten(6, 7).flatten(4, 5).flatten(2, 3)
    return latents

def _normalize_latents(
    latents: torch.Tensor, latents_mean: torch.Tensor, latents_std: torch.Tensor, scaling_factor: float = 1.0,
    reverse=False,
) -> torch.Tensor:
    # Normalize latents across the channel dimension [B, C, F, H, W]
    latents_mean = latents_mean.view(1, -1, 1, 1, 1).to(latents.device, latents.dtype)
    latents_std = latents_std.view(1, -1, 1, 1, 1).to(latents.device, latents.dtype)
    if not reverse:
        latents = (latents - latents_mean) * scaling_factor / latents_std
    else:
        latents = latents * latents_std / scaling_factor + latents_mean
    return latents

In [4]:
vid = 'B9ZQfpwK--8_000050_000060_0.0_3.5'
file = f"/mnt/sda1/saksham/TI2AV/ltx_output/audioldm_cache/{vid}.pt"
data = torch.load(file)

In [5]:
ll = data["latents"]
ll.shape

torch.Size([1, 2048, 128])

In [6]:
num_frames = 1; height = 16; width = 128
device = "cuda"; dtype = torch.bfloat16
lt = _unpack_latents(ll.to(device, dtype=dtype), 1, 16, 128)
lt = _normalize_latents(lt, vae.latents_mean, vae.latents_std, reverse=True)

In [7]:
lt.shape

torch.Size([1, 128, 1, 16, 128])

In [13]:
lt1 = lt[0].permute(1,0,3,2)
lt1 = lt1[:,:8]
lt1.shape

torch.Size([1, 8, 128, 16])

In [11]:
import sys

path = '/home/sxk230060/TI2AV/misc/ltx_lora_training_i2v_t2v/audioldm_vae/'
sys.path.append(path)
from aud_utils import AudioLDM_VAE
from IPython.display import Audio

In [15]:
vae_obj = AudioLDM_VAE()

Fetching 9 files: 100%|██████████| 9/9 [00:00<00:00, 57368.90it/s]
/home/sxk230060/miniconda3/envs/ltx_audio/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/home/sxk230060/TI2AV/misc/ltx_lora_training_i2v_t2v/audioldm/audio/stft.py:42: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = pad_center(fft_window, filter_length)
/home/sxk230060/TI2AV/misc/ltx_lora_training_i2v_t2v/audioldm/audio/stft.py:151: FutureWarning: Pass sr=16000, n_fft=1024, n_mels=64, fmin=0, fmax=8000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel_basis = librosa_mel_fn(


In [17]:
rec_audio = vae_obj.latent_to_audio(lt1.float())

In [18]:
Audio(rec_audio, rate=16000)

In [19]:
vid = 'B9ZQfpwK--8_000050_000060_0.0_3.5'
base_aud_dir = '/mnt/sda1/saksham/TI2AV/AVSync15/audios_together'
orig = os.path.join(base_aud_dir, f'{vid}.wav')
Audio(orig, rate=16000)